In order to check the effectiveness of our model, we want to now how correlated our data is to gas. To do that we want a dataframe that holds all the data in a pairwise fasion (this means that the data points line up to one another), then we can check how correlated the data is.

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import datetime


from matplotlib import pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas.tools.plotting import autocorrelation_plot
from pandas.plotting import register_matplotlib_converters

In [2]:
df_dji = pd.read_csv("data/DJI.csv") ## Reading DJI
df_sp500 = pd.read_csv('data/S&P500.csv')
file = pd.ExcelFile("data/GAS.xls")   ## Load the gas prices database
df_nas = pd.read_csv('data/NASDAQ.csv')

#Column selection
df_gas= file.parse('Data 1', header=2, ncolumns=2) # Parse the file, saving as our database
df_gas = df_gas[['Date', 'Weekly U.S. All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)']]
df_gas = df_gas.rename(index=str, columns={"Weekly U.S. All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)": "gas_price"})
df_nas = df_nas[['Date', 'Close']]
#Stocks
df_dji = df_dji[['Date', 'Close']]
df_sp500 = df_sp500[['Date', 'Close']]

#Dropping NaN values
df_gas.dropna(axis=0, inplace=True)
df_sp500.dropna(axis=0, inplace=True)
df_dji.dropna(axis=0, inplace=True)
df_nas.dropna(axis=0, inplace=True)

#Converting dates from string to datetime
df_dji['Date'] = pd.to_datetime(df_dji.Date)
df_sp500['Date'] = pd.to_datetime(df_sp500.Date)
df_gas['Date'] = pd.to_datetime(df_gas.Date)
df_nas['Date'] = pd.to_datetime(df_nas.Date)

#Set indexes
df_dji.set_index('Date', inplace = True)
df_gas.set_index('Date', inplace = True)
df_sp500.set_index('Date', inplace = True)
df_nas.set_index('Date', inplace = True)

#Rename Columns for our Master/Cross-Correlational Dataframe
df_dji = df_dji.rename(columns ={'Close': "DJI"})
df_sp500 = df_sp500.rename(columns ={'Close': "SP500"})
df_nas = df_nas.rename(columns ={'Close': "NASDAQ"})

In [3]:
df_gas.head()

,gas_price
Date,
1993-04-05,1.068
1993-04-12,1.079
1993-04-19,1.079
1993-04-26,1.086
1993-05-03,1.086


In [4]:
df_dji.head()

,DJI
Date,
1985-01-29,1292.619995
1985-01-30,1287.880005
1985-01-31,1286.770020
1985-02-01,1277.719971
1985-02-04,1290.079956


In [5]:
df_sp500.head()

,SP500
Date,
1950-01-03,16.66
1950-01-04,16.85
1950-01-05,16.93
1950-01-06,16.98
1950-01-09,17.08


In [6]:
df_nas.head()

,NASDAQ
Date,
1971-02-05,100.000000
1971-02-08,100.839996
1971-02-09,100.760002
1971-02-10,100.690002
1971-02-11,101.449997


We can now combine these dataframes into one to get how correlated they are. So we want to join the dataframes on their dates so that they line up to do a pairwise comparision. Then we want to drop any null values so that they are do not affect the analysis (they are ignored anyway when using the correlation function). 

In [7]:
df_corr_gas = pd.concat([df_gas, df_dji, df_sp500, df_nas], axis=1, join_axes=[df_gas.index])
df_corr_gas.dropna(axis=0, inplace = True)   #Drop null values
df_corr_gas.head(10)

,gas_price,DJI,SP500,NASDAQ
Date,,,,
1993-04-05,1.068,3379.189941,442.290009,670.710022
1993-04-12,1.079,3428.090088,448.369995,673.119995
1993-04-19,1.079,3466.989990,447.459991,663.030029
1993-04-26,1.086,3398.370117,433.540009,645.869995
1993-05-03,1.086,3446.459961,442.459991,666.710022
1993-05-10,1.097,3443.280029,442.799988,682.820007
1993-05-17,1.106,3449.929932,440.369995,677.960022
1993-05-24,1.106,3507.780029,448.000000,694.690002
1993-06-07,1.104,3532.129883,447.690002,694.609985


In [8]:
df_corr_gas.corr()

,gas_price,DJI,SP500,NASDAQ
gas_price,1.000000,0.583782,0.527804,0.468103
DJI,0.583782,1.000000,0.991513,0.965525
SP500,0.527804,0.991513,1.000000,0.978163
NASDAQ,0.468103,0.965525,0.978163,1.000000


This correlation table gives us the Pearson correlation coefficient (r) for each pairing. This tells us that, based on this dataframe, gas prices and the stocks are fairly correlated with DJI being the most correlated. This means that the stock options are somewhat good at predicting gas_prices. 

We now want to check the correlation between the stocks themselves. So now we join the dataframes for DJI, SP500, and NASDAQ on the dates so that they pairwise correlation. Then we again drop the null values. 

In [11]:
df_corr_stocks = pd.concat([df_dji, df_sp500, df_nas], axis=1, join_axes=[df_dji.index])
df_corr_stocks.head(10)

,DJI,SP500,NASDAQ
Date,,,
1985-01-29,1292.619995,179.179993,276.200012
1985-01-30,1287.880005,179.389999,278.600006
1985-01-31,1286.770020,179.630005,278.700012
1985-02-01,1277.719971,178.630005,278.399994
1985-02-04,1290.079956,180.350006,280.799988
1985-02-05,1285.229980,180.610001,282.600006
1985-02-06,1280.589966,180.429993,284.500000
1985-02-07,1290.079956,181.820007,287.100006
1985-02-08,1289.969971,182.190002,288.399994


In [12]:
df_corr_stocks.corr()

,DJI,SP500,NASDAQ
DJI,1.000000,0.995150,0.970717
SP500,0.995150,1.000000,0.978771
NASDAQ,0.970717,0.978771,1.000000


From this correlation table we can see that the stocks are heavily correlated with one another. With DJI and SP500 being the most correlated. 